In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Library

In [ ]:
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn import metrics

# Import Dataset

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/IYKRA/#9/Train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/IYKRA/#9/Test.csv')

In [ ]:
train_data.shape

(48, 12)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   48 non-null     object 
 1   day                    48 non-null     object 
 2   targeted_productivity  48 non-null     float64
 3   smv                    48 non-null     float64
 4   wip                    48 non-null     float64
 5   over_time              48 non-null     int64  
 6   incentive              48 non-null     int64  
 7   idle_time              48 non-null     float64
 8   idle_men               48 non-null     int64  
 9   no_of_style_change     48 non-null     int64  
 10  no_of_workers          48 non-null     float64
 11  actual_productivity    48 non-null     float64
dtypes: float64(6), int64(4), object(2)
memory usage: 4.6+ KB


In [ ]:
train_data.head()

,date,day,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Thursday,0.65,23.69,861.0,7200,0,0.0,0,0,60.0,0.521180
1,1/3/2015,Saturday,0.70,23.69,544.0,13800,0,0.0,0,0,60.0,0.700134
2,1/4/2015,Sunday,0.80,28.08,878.0,4260,50,0.0,0,0,55.5,0.800598
3,1/5/2015,Monday,0.80,28.08,759.0,6900,50,0.0,0,0,57.5,0.800077
4,1/6/2015,Tuesday,0.80,28.08,925.0,10350,50,0.0,0,0,57.5,0.800594


In [ ]:
test_data.shape

(11, 11)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   11 non-null     object 
 1   day                    11 non-null     object 
 2   targeted_productivity  11 non-null     float64
 3   smv                    11 non-null     float64
 4   wip                    11 non-null     float64
 5   over_time              11 non-null     int64  
 6   incentive              11 non-null     int64  
 7   idle_time              11 non-null     float64
 8   idle_men               11 non-null     int64  
 9   no_of_style_change     11 non-null     int64  
 10  no_of_workers          11 non-null     float64
dtypes: float64(5), int64(4), object(2)
memory usage: 1.1+ KB


# Data Preprocessing

In [ ]:
train_data.isnull().sum()

date                     0
day                      0
targeted_productivity    0
smv                      0
wip                      0
over_time                0
incentive                0
idle_time                0
idle_men                 0
no_of_style_change       0
no_of_workers            0
actual_productivity      0
dtype: int64

In [ ]:
train_data.describe()

,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
count,48.000000,48.000000,48.000000,48.0000,48.000000,48.000000,48.000000,48.000000,48.00000,48.000000
mean,0.723958,26.839167,1275.270833,7707.5000,44.979167,3.125000,0.312500,0.395833,57.37500,0.726065
std,0.109637,4.981340,3231.835351,2423.0107,27.291562,21.650635,2.165064,0.535529,1.60948,0.141864
min,0.350000,22.400000,287.000000,480.0000,0.000000,0.000000,0.000000,0.000000,51.50000,0.350633
25%,0.700000,22.520000,665.250000,6615.0000,25.250000,0.000000,0.000000,0.000000,56.50000,0.647026
50%,0.750000,28.080000,857.000000,7050.0000,50.000000,0.000000,0.000000,0.000000,57.50000,0.795003
75%,0.800000,30.100000,922.750000,10170.0000,63.000000,0.000000,0.000000,1.000000,58.50000,0.800852
max,0.800000,50.480000,23122.000000,13800.0000,94.000000,150.000000,15.000000,2.000000,60.00000,0.900800


In [ ]:
df = train_data.copy()

In [ ]:
df.head()

,date,day,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Thursday,0.65,23.69,861.0,7200,0,0.0,0,0,60.0,0.521180
1,1/3/2015,Saturday,0.70,23.69,544.0,13800,0,0.0,0,0,60.0,0.700134
2,1/4/2015,Sunday,0.80,28.08,878.0,4260,50,0.0,0,0,55.5,0.800598
3,1/5/2015,Monday,0.80,28.08,759.0,6900,50,0.0,0,0,57.5,0.800077
4,1/6/2015,Tuesday,0.80,28.08,925.0,10350,50,0.0,0,0,57.5,0.800594


In [ ]:
# day  = pd.get_dummies(df['day'])
# df_new = df.merge(day, left_index=True, right_index=True)
# df_new.head()

In [ ]:
df= df.drop(['date', 'day'], axis=1)
df.head()

,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,0.65,23.69,861.0,7200,0,0.0,0,0,60.0,0.521180
1,0.70,23.69,544.0,13800,0,0.0,0,0,60.0,0.700134
2,0.80,28.08,878.0,4260,50,0.0,0,0,55.5,0.800598
3,0.80,28.08,759.0,6900,50,0.0,0,0,57.5,0.800077
4,0.80,28.08,925.0,10350,50,0.0,0,0,57.5,0.800594


# Training Model

In [ ]:
X = df[['targeted_productivity', 'smv', 'wip', 'over_time','incentive', 'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers']]
y = df['actual_productivity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# define the base models
level0 = [('knn', KNeighborsRegressor()), ('rf', RandomForestRegressor()),
          ('svm', SVR())]
# define meta learner model
level1 = LinearRegression()
# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X_train, y_train)
# make a prediction 
predictions = model.predict(X_test)

# r2 = metrics.r2_score(y_test, predictions)
# print('R2: {}\n'.format(r2))
# mse = metrics.mean_squared_error(y_test, predictions)
# print('MSE: {}\n'.format(mse))
# print('RMSE: {}\n'.format(np.sqrt(mse)))
# mae = metrics.mean_absolute_error(y_test, predictions)
# print('MAE: {}\n'.format(mae))
mape = metrics.mean_absolute_percentage_error(y_test, predictions)
print('MAPE: {}\n'.format(mape))

MAPE: 0.0676884121072899



# Testing

In [ ]:
df_test = test_data.copy()

In [ ]:
# day_test  = pd.get_dummies(df_test['day'])
# df_test = df_test.merge(day_test, left_index=True, right_index=True)
# df_test.head()

In [ ]:
df_test = df_test.drop(['date', 'day'], axis=1)

In [ ]:
predictions_new = model.predict(df_test)

In [ ]:
predictions_new

array([0.54525407, 0.49922483, 0.49777658, 0.62097247, 0.68946861,
       0.70277835, 0.69971011, 0.70449462, 0.76659248, 0.78486562,
       0.84057274])

In [ ]:
Id = range(1,(df_test.shape[0]+1))
print(Id)
submission = pd.DataFrame(Id, columns = ['Id'])
submission['actual_productivity']=predictions_new

range(1, 12)


In [ ]:
submission.to_csv('/content/drive/MyDrive/IYKRA/#9/sub2.csv',index=False)